Mooncoin Market Analysis
========================

Documentation:
- https://github.com/ccxt/ccxt/wiki

In [1]:
import ccxt
from pydash import py_ as _
from datetime import datetime, timedelta
from dateutil.parser import parse
import pandas as pd
import datetime
import numpy as np
import matplotlib
import re

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 8)

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [2]:
bleutrade    = ccxt.bleutrade({"timeout": 100*1000})
bittrex      = ccxt.bittrex()

In [3]:
moonbtc_trades = pd.DataFrame(bleutrade.fetch_trades("MOON/BTC", params={"count": 200}), columns=['amount', 'datetime', 'side', 'info'])
moonbtc_trades['price']      = moonbtc_trades['info'].map(lambda x: float(x['Price']))
moonbtc_trades['btc_amount'] = moonbtc_trades['amount'] * moonbtc_trades['price']
del moonbtc_trades['info']
moonbtc_trades['datetime'] = pd.to_datetime(moonbtc_trades['datetime']) # convert seconds to nanoseconds
moonbtc_trades

           amount            datetime side         price  btc_amount
0    1.000000e+04 2018-01-02 01:37:23  buy  3.000000e-08    0.000300
1    5.722657e+05 2018-01-02 01:37:20  buy  3.000000e-08    0.017168
2    4.123824e+04 2018-01-02 01:34:11  buy  3.000000e-08    0.001237
3    1.000000e+04 2018-01-02 01:33:25  buy  3.000000e-08    0.000300
..            ...                 ...  ...           ...         ...
196  5.192153e+04 2018-01-01 22:47:42  buy  3.000000e-08    0.001558
197  9.592678e+04 2018-01-01 22:46:05  buy  3.000000e-08    0.002878
198  1.000000e+07 2018-01-01 22:46:05  buy  3.000000e-08    0.300000
199  1.000000e+07 2018-01-01 22:46:05  buy  3.000000e-08    0.300000

[200 rows x 5 columns]

Mooncoin Volumes
----------------

**How far back in time does bleutrade order moonbtc_trades go?**

- 2017-12-31: for max moonbtc_trades of 200 orders, Timedelta('0 days 11:28:11'), which is less than a day

In [4]:
moonbtc_trades['datetime'].max() - moonbtc_trades['datetime'].min()

Timedelta('0 days 02:51:18')

**What is the average order size?**

- 2017-12-31: Average orders are BTC 0.006385 (buy@2) - 0.007357 (sell@1) which is about $85-100 USD
- 2017-12-31: Average orders are BTC 0.034786 (buy@2) - 0.000630 (sell@1) which is an order of magnitude shift in both directions

In [5]:
moonbtc_trades.groupby('side').mean()

            amount         price  btc_amount
side                                        
buy   1.330161e+06  3.000000e-08    0.039905
sell  2.408632e+06  2.000000e-08    0.048173

**What is the relative volume on the buy and sell sides of the market**

- 2017-12-31: There is currently 2.9x BTC / 5.8x MOON volume on the sell@1 side of the market
- 2018-01-01: There is currently 3.4 BTC / 6.8x MOON volume on the buy@2 side of the market

In [6]:
moonbtc_trades_sum = moonbtc_trades[['amount', 'btc_amount','side']].groupby('side').sum()
moonbtc_trades_sum

            amount  btc_amount
side                          
buy   2.407592e+08    7.222776
sell  4.576400e+07    0.915280

In [7]:
moonbtc_trades_sum.loc['sell'] / moonbtc_trades_sum.loc['buy']

amount        0.190082
btc_amount    0.126721
dtype: float64

**Can we plot a chart for the buy/sell volume history**

- 2017-12-31 - Apart from a massive 0.9 BTC sell@1 spike at 7am, volumes remain low but about even on both sides of the market
- 2018-01-01 - Massive 5 BTC buy@2 spike at 10am

In [8]:
moonbtc_trades_agg = moonbtc_trades.groupby([moonbtc_trades['datetime'].dt.hour, 'side'])['btc_amount'].sum()
moonbtc_trades_agg.unstack().plot()
moonbtc_trades_agg

datetime  side
0         buy     2.455956
          sell    0.017669
1         buy     0.336713
          sell    0.007456
22        buy     1.681585
23        buy     2.748522
          sell    0.890154
Name: btc_amount, dtype: float64

Price
-----

** What is the price of mooncoin **

In [9]:
tickers = {
    'bleutrade': bleutrade.fetch_tickers(),
    'bittrex':   bittrex.fetch_tickers(),
}
tickers['bleutrade']['MOON/BTC']

{'ask': 3e-08,
 'average': None,
 'baseVolume': 3398190102.425489,
 'bid': 2e-08,
 'change': None,
 'close': None,
 'datetime': '2018-01-02T01:39:06.000Z',
 'first': None,
 'high': 3e-08,
 'info': {'Ask': '0.00000003',
  'Average': '0.00000002',
  'BaseCurrency': 'Bitcoin',
  'BaseVolume': '62.77901655',
  'Bid': '0.00000002',
  'High': '0.00000003',
  'IsActive': 'true',
  'Last': '0.00000003',
  'Low': '0.00000001',
  'MarketCurrency': 'Mooncoin',
  'MarketName': 'MOON_BTC',
  'PrevDay': '0.00000002',
  'TimeStamp': '2018-01-02 01:39:06',
  'Volume': '3398190102.42548895'},
 'last': 3e-08,
 'low': 1e-08,
 'open': None,
 'percentage': None,
 'quoteVolume': 62.77901655,
 'symbol': 'MOON/BTC',
 'timestamp': 1514857146000,
 'vwap': None}

In [10]:
def bid(market):       return market['bid']
def mid(market):       return (market['ask'] + market['bid'])/2
def spread(market):    return (market['ask'] - market['bid'])
def spread_pc(market): return spread(market)/mid(market)

In [11]:

price = {'bittrex': {}, 'bleutrade': {}}
price['bittrex']['BTC/USDT']    = bid(tickers['bittrex']['BTC/USDT'])
price['bittrex']['DOGE/BTC']    = bid(tickers['bittrex']['DOGE/BTC'])
price['bittrex']['ETH/BTC']     = bid(tickers['bittrex']['ETH/BTC'])
price['bleutrade']['DOGE/BTC']  = bid(tickers['bleutrade']['DOGE/BTC'])
price['bleutrade']['ETH/BTC']   = bid(tickers['bleutrade']['ETH/BTC'])
price['bleutrade']['MOON/BTC']  = bid(tickers['bleutrade']['MOON/BTC'])
price['bleutrade']['MOON/DOGE'] = bid(tickers['bleutrade']['MOON/DOGE'])
price['bleutrade']['MOON/ETH']  = bid(tickers['bleutrade']['MOON/ETH']) 

price['bleutrade']['MOON/ETH/BTC']     = price['bleutrade']['MOON/ETH']  * price['bleutrade']['ETH/BTC']
price['bleutrade']['MOON/DOGE/BTC']    = price['bleutrade']['MOON/DOGE'] * price['bleutrade']['DOGE/BTC']
price['bittrex']['MOON/ETH/BTC']       = price['bleutrade']['MOON/ETH']  * price['bittrex']['ETH/BTC']
price['bittrex']['MOON/DOGE/BTC']      = price['bleutrade']['MOON/DOGE'] * price['bittrex']['DOGE/BTC']
price['bittrex']['MOON/ETH/BTC/USDT']  = price['bleutrade']['MOON/ETH/BTC']  * price['bittrex']['BTC/USDT']
price['bittrex']['MOON/DOGE/BTC/USDT'] = price['bleutrade']['MOON/DOGE/BTC'] * price['bittrex']['BTC/USDT']
price['bittrex']['MOON/BTC/USDT']      = price['bleutrade']['MOON/BTC']      * price['bittrex']['BTC/USDT']                                                                                         

price                                                                                              

{'bittrex': {'BTC/USDT': 13706.2631579,
  'DOGE/BTC': 6.4e-07,
  'ETH/BTC': 0.05930124,
  'MOON/BTC/USDT': 0.000274125263158,
  'MOON/DOGE/BTC': 2.5919929599999997e-08,
  'MOON/DOGE/BTC/USDT': 0.00035526537613184166,
  'MOON/ETH/BTC': 2.78715828e-08,
  'MOON/ETH/BTC/USDT': 0.00037649469600493934},
 'bleutrade': {'DOGE/BTC': 6.4e-07,
  'ETH/BTC': 0.05844427,
  'MOON/BTC': 2e-08,
  'MOON/DOGE': 0.04049989,
  'MOON/DOGE/BTC': 2.5919929599999997e-08,
  'MOON/ETH': 4.7e-07,
  'MOON/ETH/BTC': 2.74688069e-08}}

In [12]:
bleutrade_markets = pd.DataFrame(bleutrade.fetch_tickers()).transpose()[['ask', 'bid', 'last']]
bleutrade_markets['mid']    = (bleutrade_markets['ask'] + bleutrade_markets['bid']) / 2
bleutrade_markets['spread'] = (bleutrade_markets['ask'] - bleutrade_markets['bid']) / bleutrade_markets['bid']
bleutrade_markets = bleutrade_markets.filter(regex="MOON/|ETH/|DOGE/", axis=0)
bleutrade_markets

                 ask        bid       last        mid       spread
DOGE/BTC     6.5e-07    6.4e-07    6.4e-07   6.45e-07     0.015625
DOGE/ETH   1.295e-05  1.093e-05  1.299e-05  1.194e-05     0.184812
ETH/BTC     0.058796  0.0584443  0.0584443  0.0586201   0.00601838
ETH/DOGE      109000    83625.7    83625.7    96312.9     0.303427
MOON/BTC       3e-08      2e-08      3e-08    2.5e-08          0.5
MOON/DOGE  0.0404999  0.0404999  0.0404999  0.0404999  2.46914e-07
MOON/ETH     4.8e-07    4.7e-07    4.9e-07   4.75e-07    0.0212766

In [13]:
for symbol in bleutrade_markets.transpose():
    top  = re.split('/', symbol)[0]
    base = re.split('/', symbol)[-1]
    if base == 'BTC': continue
    if base in ['ETH', 'DOGE']:
        bleutrade_markets.loc[symbol+'/BTC'] = bleutrade_markets.loc[symbol] * bleutrade_markets.loc[base+'/BTC'] 
        bleutrade_markets.loc[symbol+'/BTC']['spread'] = bleutrade_markets.loc[symbol]['spread'] + bleutrade_markets.loc[base+'/BTC']['spread']         

print_full(bleutrade_markets)

                       ask          bid         last          mid       spread
DOGE/BTC           6.5e-07      6.4e-07      6.4e-07     6.45e-07     0.015625
DOGE/ETH         1.295e-05    1.093e-05    1.299e-05    1.194e-05     0.184812
ETH/BTC           0.058796    0.0584443    0.0584443    0.0586201   0.00601838
ETH/DOGE            109000      83625.7      83625.7      96312.9     0.303427
MOON/BTC             3e-08        2e-08        3e-08      2.5e-08          0.5
MOON/DOGE        0.0404999    0.0404999    0.0404999    0.0404999  2.46914e-07
MOON/ETH           4.8e-07      4.7e-07      4.9e-07     4.75e-07    0.0212766
DOGE/ETH/BTC   7.61408e-07  6.38796e-07  7.59191e-07  6.99924e-07     0.190831
ETH/DOGE/BTC       0.07085    0.0535205    0.0535205    0.0621218     0.319052
MOON/DOGE/BTC  2.63249e-08  2.59199e-08  2.59199e-08  2.61224e-08    0.0156252
MOON/ETH/BTC   2.82221e-08  2.74688e-08  2.86377e-08  2.78446e-08     0.027295


All MoonCoin Markets
--------------------

In [14]:
tickers = {}
moon_exchanges = 'bleutrade'
for exchange_name in list(moon_exchanges or ccxt.exchanges):
    try: 
        ticker  = getattr(ccxt, exchange_name)().fetch_tickers()
        symbols = list(filter(lambda symbol: "MOON" in symbol, ticker.keys()))
        if len(symbols):            
            if not exchange_name in moon_exchanges: moon_exchanges += [exchange_name]
            for symbol in symbols:
                ticker[symbol]['exchange'] = exchange_name
                tickers[exchange_name+':'+symbol] = ticker[symbol]
    except: pass
        
moon_exchanges

'bleutrade'

In [15]:
tickers_df = pd.DataFrame(tickers)
tickers_df.transpose()

Empty DataFrame
Columns: []
Index: []